In [2]:
import pandas as pd
import geopandas as gp

In [3]:
data = pd.read_csv('mbta_ridership.csv')
data.head()

,mode,season,route_id,route_name,direction_id,day_type_id,day_type_name,time_period_id,time_period_name,stop_name,stop_id,total_ons,total_offs,number_service_days,average_ons,average_offs,average_flow,ObjectId
0,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Allston Street,place-alsgr,0,17,77,0,0,4,1
1,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Arlington,place-armnl,2675,8021,77,35,104,381,2
2,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Babcock Street,place-babck,0,151,77,0,2,8,3
3,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Back of the Hill,place-bckhl,0,36,77,0,0,4,4
4,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Beaconsfield,place-bcnfd,12,67,77,0,1,44,5
